In [20]:
import torch
import glob

def data_sampler(path_1, path_2):
    #path_2 = r'C:\Users\wtana\Desktop\LiDAR_cm\syoumen\kaneko\200cm\txt\*'
    true_files = glob.glob(path_1)
    false_files = glob.glob(path_2)
    
    len_true = len(true_files)
    len_false = len(false_files)
    if len_true == 0 or len_false == 0:
        print('No data available')
        return 0,0

    true_list = []
    false_list = []

    for idx in range(len_true):
        s_num = []
        with open(true_files[idx],'r') as f:
            data = f.read().split('\n')
            for i in range(len(data)):
                if data[i] == '':
                    continue
                s = data[i].split(' ')
                s_num.append([float(s[0]),float(s[1]),float(s[2])])
        true_list.append([s_num])

    for idx in range(len_false):
        s_num = []
        with open(false_files[idx],'r') as f:
            data = f.read().split('\n')
            for i in range(len(data)):
                if data[i] == '':
                    continue
                s = data[i].split(' ')
                s_num.append([float(s[0]),float(s[1]),float(s[2])])
        false_list.append([s_num])
    len_true = len(true_list)
    len_false = len(false_list)

    true_labels = torch.ones(len(true_list))
    false_labels = torch.zeros(len(false_list))

    #return true_list,false_list

    input_data = torch.cat((torch.Tensor(true_list),torch.Tensor(false_list)))
    labels = torch.cat((torch.Tensor(true_labels), torch.Tensor(false_labels)))
    return input_data.view(-1,3), labels.view(-1, 1)



In [2]:
from model import *
#from sampler import *
#from self_made import *

loss = []
Accu = []

batch_size = 64
num_points = 64
num_labels = 1
def main_():
    pointnet = PointNet(num_points, num_labels)

    new_param = pointnet.state_dict()
    new_param['main.0.main.6.bias'] = torch.eye(3, 3).view(-1)
    new_param['main.3.main.6.bias'] = torch.eye(64, 64).view(-1)
    pointnet.load_state_dict(new_param)

    criterion = nn.BCELoss()
    optimizer = optim.Adam(pointnet.parameters(), lr=0.001)

    loss_list = []
    accuracy_list = []

    for iteration in range(200):

        pointnet.zero_grad()
        input_data, labels = data_sampler(path_1='./normalize_txt/takuya/150cm/*',path_2='./normalize_txt/wall/*')
        #input_data, labels = data_sampler(batch_size, num_points)

        output = pointnet(input_data)
        output = nn.Sigmoid()(output)

        error = criterion(output, labels)
        error.backward()

        optimizer.step()

        with torch.no_grad():
            output[output > 0.5] = 1
            output[output < 0.5] = 0
            accuracy = (output==labels).sum().item()/batch_size

        loss_list.append(error.item())
        accuracy_list.append(accuracy)
        loss.append(float(error.item()))
        Accu.append(float(accuracy))
        if iteration % 10 == 0:

            print('Iteration : {}   Loss : {}'.format(iteration, error.item()))
            print('Iteration : {}   Accuracy : {}'.format(iteration, accuracy))
            

In [ ]:
main_()

In [21]:
input_data, labels = data_sampler(path_1='./normalize_txt/takuya/150cm/*',path_2='./normalize_txt/wall/*')

ValueError: expected sequence of length 100 at dim 2 (got 99)

In [23]:
print(len(input_data[1][0]))
print(len(labels[1][0]))


100
100


In [24]:
for i in range(len(input_data)):
    if not len(input_data[i][0]) == len(labels[i][0]):
        print(i)

3
5
6
7
8
9
10
13
15
18
19
20
22
23
24
25
26
28
29
30
31
34
35
36
37
38
39
40
41
43
45
47
48
49
50
52
53
55
57
58
60
61
63
64
65
67
68
69
70
71
75
76
77
78
79
80
81
82
84
85
88
89
90
92
93
94
95
96
98
99
100
104
106
108
109
111
114
115
116
117
119
120
121
122
123
125
127
128
129
130
132
133
136
137
138
139
140
141
142
143
144
145
146
147
149
